In [ ]:
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix

folder = '/home/wilkinsonlab/Escritorio/elsevier/statements/'

INPUT_FILE = folder+"st_45_ground_truth_participants_3l.csv"

texts_test = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels_test = []  # list of label ids
fin = codecs.open(INPUT_FILE,"r",  encoding='utf8')
for line in fin:
    sent, certain = line.strip().split("\t")
    sent = [x for x in nltk.word_tokenize(sent) if x not in stopwords]
    texts_test.append(' '.join(sent))
    labels_test.append(certain)

    
print(len(texts_test))
print(len(labels_test))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts_test)

texts_test = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
labels_test = np.asarray(labels_test, dtype=int)
labels_test = list(labels_test)
for index, item in enumerate(labels_test):
    if item == 1:
        labels_test[index] = [1,0,0]
    elif item == 2:
        labels_test[index] = [0,1,0]
    else:
        labels_test[index] = [0,0,1]
labels_test = np.asarray(labels_test)
# labels_test

In [ ]:
VALIDATION_SPLIT = 0.05
MAX_NB_WORDS = 6660
EMBEDDING_DIM = 200

In [ ]:
import numpy as np
import os

embeddings_index = {}
with open(os.path.join("/home/wilkinsonlab/Escritorio/elsevier/", 'dictionary-200.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Indexing word vectors.')

In [ ]:
import codecs
import nltk
import np_utils
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D, Flatten
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras.layers.pooling import GlobalMaxPooling1D, MaxPooling1D
from keras.layers import LSTM, Bidirectional, Concatenate, GRU, CuDNNGRU, CuDNNLSTM
from keras.layers.merge import concatenate
from sklearn.model_selection import StratifiedKFold
from keras.layers import Embedding
from keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
import pickle
import matplotlib.pyplot as plt
# from pylab import figure

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

puntuacion = 0.80
folder = '/home/wilkinsonlab/Escritorio/elsevier/statements/'

INPUT_FILE = folder+"st_3221_ground_truth_train+val_3l.csv"

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
fin = codecs.open(INPUT_FILE,"r",  encoding='utf8')
maxlen = 0
for line in fin:
    sent, certain = line.strip().split("\t")
    sent = [x for x in nltk.word_tokenize(sent) if x not in stopwords]
    texts.append(' '.join(sent))
    labels.append(certain)

    if len(sent) > maxlen:
        maxlen = len(sent)
fin.close()   
print(len(texts))
print(len(labels))
MAX_SEQUENCE_LENGTH = maxlen

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
labels = np.asarray(labels, dtype=int)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
labels

In [ ]:
kfold = StratifiedKFold(n_splits=20, shuffle=True)
cvscores = []

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
cvscores = []
np.random.seed(1)
historial = []
train_test= []
contador = 0
for train, test in kfold.split(data, labels):
    if contador == 0:
        labels = list(labels)
        for index, item in enumerate(labels):
            if item == 1:
                labels[index] = [1,0,0]
            elif item == 2:
                labels[index] = [0,1,0]
            else:
                labels[index] = [0,0,1]
        labels = np.asarray(labels)
        contador = contador + 1
    train_test.append([train,test])
    #     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, mode='auto', patience=3, min_lr=0.002)
    stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=1, mode='auto', restore_best_weights=True)
    model = Sequential()
    model.add(Embedding(num_words, EMBEDDING_DIM, 
                            input_length=MAX_SEQUENCE_LENGTH))
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False
    model.add(Dense(300,kernel_regularizer=regularizers.l1(0.000001)))
    model.add(Dropout(0.35))
    model.add(Dense(200,kernel_regularizer=regularizers.l1(0.000001)))
    model.add(Dropout(0.35))
    model.add(Dense(100,kernel_regularizer=regularizers.l1(0.000001)))
    model.add((LSTM(50, dropout=0.1, recurrent_dropout=0.1)))#, kernel_regularizer=regularizers.l1(0.00001))))
    model.add(Dense(3,kernel_regularizer=regularizers.l1(0.000001)))
    model.add(Activation("sigmoid"))

    model.compile(loss="binary_crossentropy", optimizer="rmsprop", 
                    metrics=["accuracy"])

    history = model.fit(data[train], labels[train], batch_size=20, 
                            epochs=10
                           ,validation_data=(data[test], labels[test]), callbacks=[stop])
    historial.append(history)
    # evaluate model
    scores = model.evaluate(data[test], labels[test], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("Test score: {:.3f}, accuracy: {:.3f}".format(scores[0], scores[1]))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print cvscores